<a href="https://colab.research.google.com/github/HimanshSRM/Ratings-And-Reviews/blob/main/bert_fake_review_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch datasets scikit-learn pandas numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.9/664.8 MB 120.8 MB/s eta 0:00:05
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd

# Load dataset (Update path if needed)
file_path = "/content/fake reviews dataset.csv"  # Change path if file is elsewhere
df = pd.read_csv(file_path)

# Show basic information
print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40432 entries, 0 to 40431
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   category  40432 non-null  object 
 1   rating    40432 non-null  float64
 2   label     40432 non-null  object 
 3   text_     40432 non-null  object 
dtypes: float64(1), object(3)
memory usage: 1.2+ MB
None
             category  rating label  \
0  Home_and_Kitchen_5     5.0    CG   
1  Home_and_Kitchen_5     5.0    CG   
2  Home_and_Kitchen_5     5.0    CG   
3  Home_and_Kitchen_5     1.0    CG   
4  Home_and_Kitchen_5     5.0    CG   

                                               text_  
0  Love this!  Well made, sturdy, and very comfor...  
1  love it, a great upgrade from the original.  I...  
2  This pillow saved my back. I love the look and...  
3  Missing information on how to use it, but it i...  
4  Very nice set. Good quality. We have had the s...  


In [3]:
import re
from sklearn.model_selection import train_test_split

# Rename for clarity
df = df.rename(columns={'text_': 'review_text', 'label': 'label'})

# Convert labels (1 = Fake, 0 = Real)
df['label'] = df['label'].apply(lambda x: 1 if x == 'CG' else 0)

# Function to clean text
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.lower().strip()  # Lowercase and remove spaces
    return text

df['cleaned_text'] = df['review_text'].apply(clean_text)

# Split into training/testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['cleaned_text'], df['label'], test_size=0.2, random_state=42
)

print(f"Training samples: {len(train_texts)}, Testing samples: {len(test_texts)}")


Training samples: 32345, Testing samples: 8087


In [4]:
from transformers import BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize reviews
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=512, return_tensors="pt")
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=512, return_tensors="pt")

# Check tokenized shape
print(train_encodings['input_ids'].shape, test_encodings['input_ids'].shape)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

torch.Size([32345, 512]) torch.Size([8087, 512])


In [5]:
import torch

class FakeReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Create dataset objects
train_dataset = FakeReviewDataset(train_encodings, train_labels.values)
test_dataset = FakeReviewDataset(test_encodings, test_labels.values)


In [6]:
from transformers import BertForSequenceClassification

# Load model with 2 output labels (Fake/Real)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
!pip install --upgrade transformers # Upgrade transformers to the latest version

In [13]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    # The 'evaluation_strategy' argument was introduced in a later version.
    # For older versions, you might need to omit it or use a different approach
    # for evaluation. If you're using a recent version and still getting this error,
    # make sure your transformers library is up-to-date.
    # evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [14]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: himanshyadav202 (himanshyadav202-srm-institute-of-science-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.345700
1000,0.240900
1500,0.181900
2000,0.148800
2500,0.172300
3000,0.145700
3500,0.115100
4000,0.147000
4500,0.069800
5000,0.069500


TrainOutput(global_step=12132, training_loss=0.08866275441634784, metrics={'train_runtime': 9155.4452, 'train_samples_per_second': 10.599, 'train_steps_per_second': 1.325, 'total_flos': 2.55309812568576e+16, 'train_loss': 0.08866275441634784, 'epoch': 3.0})

In [15]:
from google.colab import drive
drive.mount('/content/drive')

# Save model
model.save_pretrained('/content/drive/MyDrive/bert_fake_review_model')
tokenizer.save_pretrained('/content/drive/MyDrive/bert_fake_review_model')


Mounted at /content/drive


('/content/drive/MyDrive/bert_fake_review_model/tokenizer_config.json',
 '/content/drive/MyDrive/bert_fake_review_model/special_tokens_map.json',
 '/content/drive/MyDrive/bert_fake_review_model/vocab.txt',
 '/content/drive/MyDrive/bert_fake_review_model/added_tokens.json')

In [16]:
from transformers import BertForSequenceClassification, BertTokenizer

model_path = "/content/drive/MyDrive/bert_fake_review_model"
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)


In [17]:
def predict_fake_review(review_text):
    inputs = tokenizer(review_text, truncation=True, padding=True, max_length=512, return_tensors="pt")
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return "Fake Review" if prediction == 1 else "Real Review"

# Example
review = "This product is amazing! Best purchase ever!"
print(predict_fake_review(review))


Real Review


In [25]:
def predict_fake_review(review_text):
    inputs = tokenizer(review_text, truncation=True, padding=True, max_length=512, return_tensors="pt")
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return "Fake Review" if prediction == 1 else "Real Review"

# Example
review = "Amazing purchase !"
print(predict_fake_review(review))


Fake Review
